In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [2]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1.eval'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
p2t_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
p2t_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample.eval'
# context model
context_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
context_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'

In [3]:
df = pd.read_feather('../working/train.fea')

In [4]:
df.columns

Index(['cell_id', 'cell_type', 'source', 'id', 'n_words', 'cid', 'ancestor_id',
       'parent_id', 'n_cell', 'n_code_cell', 'n_markdown_cell',
       'markdown_frac', 'rank', 'code_rank', 'markdown_rank', 'rel_rank',
       'pct_rank', 'fold', 'worker'],
      dtype='object')

In [5]:
d = df[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank']]
d

,id,cell_id,rank,cell_type,rel_rank
0,00001756c60be8,1862f0a6,0,code,0.0323
1,00001756c60be8,448eb224,1,markdown,0.0484
2,00001756c60be8,2a9e43d6,2,code,0.0645
3,00001756c60be8,7e2f170a,3,markdown,0.0806
4,00001756c60be8,038b763d,4,code,0.0968
...,...,...,...,...,...
6370641,fffe1d764579d5,5369934b,67,markdown,0.9597
6370642,fffe1d764579d5,ac9db030,68,code,0.9677
6370643,fffe1d764579d5,a8ffc8b4,69,markdown,0.9758
6370644,fffe1d764579d5,70455170,70,code,0.9839


In [6]:
xp = gezi.load(f'{root}/{context_model}/valid.pkl')
dxp = pd.DataFrame(gezi.batch2list(xp))

In [7]:
# dxp

In [8]:
calc_metric(xp)

0.9063361261657104

In [9]:
calc_metric(xp, 'reg_pred')

0.9063009326627349

In [10]:
scores = calc_metrics(xp, 'reg_pred')

calc_metrics:   0%|          | 0/27590 [00:00<?, ?it/s]

In [11]:
dxp = to_df(xp, groupby=False, key='reg_pred')

In [12]:
# dxp

In [13]:
dxp = dxp[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank', 'pred']]

In [14]:
scores = scores.sort_values('score')

In [15]:
dxp.loc[dxp["cell_type"] == 'code',
        'pred'] = dxp.loc[dxp["cell_type"] == 'code', 'rel_rank']

In [16]:
def vis(idx):
  # print(scores.score.values[idx])
  id = scores.id.values[idx]
  id = '4562d150cadf6a'
  id = '1a7d96060f8e36'
  id = '674983f1fb6a94'
  d2 = dxp[dxp.id==id]
  ic(scores[scores.id==id])
  gezi.plot.display_side_by_side((d[d.id==id], 'gt'), (d2.sort_values('pred'), 'pred'))
  d_ = df[['id', 'cell_id', 'rank', 'cell_type', 'rel_rank', 'source']]
  d_ = d_[d_.id==id]
  display(d_)

In [17]:
vis(1000)

[08/06/22 17:15:41] 3503898657.py:8 in vis()
                    scores[scores.id==id]:                    id  score
                                           21671  674983f1fb6a94 0.8192


,id,cell_id,rank,cell_type,rel_rank
2566937,674983f1fb6a94,d2f7a30b,0,markdown,0.0063
2566938,674983f1fb6a94,85e8177e,1,markdown,0.0125
2566939,674983f1fb6a94,b2a35f6e,2,markdown,0.0188
2566940,674983f1fb6a94,59d81c97,3,markdown,0.0250
2566941,674983f1fb6a94,96550e11,4,code,0.0312
2566942,674983f1fb6a94,0985652f,5,code,0.0625
2566943,674983f1fb6a94,34c9255c,6,markdown,0.0781
2566944,674983f1fb6a94,5bfe2cec,7,code,0.0938
2566945,674983f1fb6a94,7094dd2a,8,code,0.1250
2566946,674983f1fb6a94,29f1b85a,9,markdown,0.1406


,id,cell_id,rank,cell_type,rel_rank,source
2566937,674983f1fb6a94,d2f7a30b,0,markdown,0.0063,"# What Makes a Second-Half Team?\n### Developing a ""Resilience"" Metric for NCAA Basketball\n### Henry Tessier"
2566938,674983f1fb6a94,85e8177e,1,markdown,0.0125,"<div align=""center""><img src=""https://lh3.googleusercontent.com/botptFTCHwUBMYFb-oFDGhJNBF73_DEI7MfLUHhshqbMH_4aQn9EmR6KryBGu0ltka42aNUspq7T28AbMlg8EdwwDzsIKj9-o3pwBCtlL8Z6nNidnvoAxTjU055KSqqaAFVBEWYOVF3q7Q07-rQzM0LOAEgyFHdp4Jg2W7O-Hyywq5HHUwOxkCzo49lCTUBIRyPL_Jc7-ZPcnsAmEO9nntsCqJyNIFixQHkXCXTAoFtw1Zze3e3dIAHKrbsAGOySG4avW656VELM3xykNkBJ33jKGirQI4SxQ864I523_64wldNGptPzAN2TK2yWgJYCCH-ty4txrXITWljB3OO6YPioeVEjVpYrnnfs-62jFGf95tPu7B4ZfkiET2Xb9TZTKkuzhim7lswZqOW9nsD7WmyAVbS_u5bX6fWPrMUce5qVPR6a..."
2566939,674983f1fb6a94,b2a35f6e,2,markdown,0.0188,"# 1. Introduction\n\nIn the world of sports analytics there are thousands of metrics to quantify the skill level of a team. From the work of Ken Pomeroy, Bart Torvik, Kenneth Massey, Jeff Sagarin, and the tons of other basketball statheads, the art of determining the better basketball team has been nearly perfected. At this point, the only way betting lines are going to get more accurate is if we start collecting stats on what players ate for lunch on gameday. \n\nWhen it comes to March Madn..."
2566940,674983f1fb6a94,59d81c97,3,markdown,0.0250,"# 2. Resilience Summary\n\nThe resilience metric is independent of team skill and strength-of-schedule, meaning that it is unaffected by the overall performance of a team as reflected by its point differential in games. Maintaining this independence ensures that resilience truly measures a unique feature of team performance not already explained by its own skill level or the skill level of its opponents.\n\n**In this analysis, it is determined that resilience has no use as a pre-game predict..."
2566941,674983f1fb6a94,96550e11,4,code,0.0312,"#imports\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport numpy as np\nimport statsmodels.api as sm\nfrom scipy.stats import ttest_rel, ttest_ind, chi2_contingency\nfrom scipy import stats\npd.options.display.max_rows = 500\nplt.style.use('fivethirtyeight')"
2566942,674983f1fb6a94,0985652f,5,code,0.0625,"#create TeamID to name dictionary\nteam_names = pd.read_csv('../input/march-madness-analytics-2020/MDataFiles_Stage2/MTeams.csv')\nteam_names = team_names[['TeamName','TeamID']]\nteam_dict = dict(zip(team_names.TeamID, team_names.TeamName))"
2566943,674983f1fb6a94,34c9255c,6,markdown,0.0781,"# 3. Data\n\nTo create the resilience metric, we rely solely on play-by-play data for each season. This data contains the actions that occurred across all games throughout the season. Among the fields included in the data are the season, winning team ID, and losing team ID, which together are sufficient to identify each unique game, as well as the score at the time of the event, the number of elapsed seconds that have occurred in the game at the time of the event, and the type of event that ..."
2566944,674983f1fb6a94,5bfe2cec,7,code,0.0938,"def clean_plays(playframe, team_dict):\n """"""Clean play by play dataframe for entire season""""""\n \n df = playframe.copy()\n \n #map team IDs to team names\n df.WTeamID = df.WTeamID.map(team_dict)\n df.LTeamID = df.LTeamID.map(team_dict)\n df.EventTeamID = df.EventTeamID.map(team_dict)\n \n #filter unnecessray columns\n df = df[['EventID','Season','DayNum','WTeamID','LTeamID','WCurrentScore','LCurrentScore','ElapsedSeconds','EventType']]\n \n #remove all ..."
2566945,674983f1fb6a94,7094dd2a,8,code,0.1250,plays20.EventType.value_counts().plot(kind='bar')\nplt.title('2019-2020 Season Event Counts by Type')\nplt.show()
2566946,674983f1fb6a94,29f1b85a,9,markdown,0.1406,"From the figure above we observe that the bulk of event types are non-scoring events such as substitutions, rebounds, and fouls. Because we are only interested in calculating point differentials, we can discard all non-scorin

In [20]:
d[d.id=='00001756c60be8']

,id,cell_id,rank,cell_type,rel_rank
0,00001756c60be8,1862f0a6,0,code,0.0323
1,00001756c60be8,448eb224,1,markdown,0.0484
2,00001756c60be8,2a9e43d6,2,code,0.0645
3,00001756c60be8,7e2f170a,3,markdown,0.0806
4,00001756c60be8,038b763d,4,code,0.0968
5,00001756c60be8,77e56113,5,markdown,0.1129
6,00001756c60be8,2eefe0ef,6,code,0.1290
7,00001756c60be8,1ae087ab,7,markdown,0.1452
8,00001756c60be8,0beab1cd,8,code,0.1613
9,00001756c60be8,8ffe0b25,9,markdown,0.1774


In [21]:
len(d[(d.id=='00001756c60be8') & (d.cell_type=='code')])

30

In [22]:
1 / 31

0.03225806451612903